Assignment: Homework 1 - Problem Solving\
Name: Won Seok Park\
Date: Feb/27/2020\
Instructor: Keith Vander Linden\
Course: CS 344

### Question 1
Inspection: the examination of one's own conscious thoughts and feelings. In psychology, the process of introspection relies exclusively on observation of one's mental state, while in a spiritual context it may refer to the examination of one's soul. Introspection is closely related to human self-reflection and self-discovery and is contrasted with external observation. 

Yes, I believe that introspection would be a useful tool for developing more human-like AI. For many decades, AI researcher focused on the optimization and problem solving as a primary attribute of AI. However, despite AI can start to outperform human in various areas, it still does not reason like humans. In the distant future, researchers may be able to fully understand brain and replicate human conciousness. However, neuroscience is nowhere close to that point. Until that day comes, I believe that self-reflection (introspection) is the best tools we've got as a heuristics that AI researchers could use to emulate human conciousness and prevent AI from becoming too distanced from our goal.


## Course Scheduling Constraint Satisfaction Problem Formulation:

Much of the barebone came from the csp.Zebra() class with some modification from Prof. VanderLinden's advice during the class.

### Note
I've used Ian Park's package to set up virtualenv.\
Import might need a tweak for the code to run on the pycharm

### Result
**Depth-first:** Took longer than 5+ minutes to complete (aborted)

**AC3:** Consistently failed (5 attempts)

**Backtracking:** Found solution

    Solution: {'cs108': 'adams mwf9:00-9:50 nh253', 'cs112': 'adams mwf10:30-11:20 nh253', 'cs212': 'adams mwf11:30-12:20 nh253', 'cs214': 'adams mwf12:30-1:20 nh253', 'cs232': 'adams mwf1:30-2:20 nh253', 'cs262': 'vanderlinden mwf9:00-9:50 sb382', 'cs344': 'vanderlinden mwf10:30-11:20 sb382'}

    Time taken to find solution: 0.0010242462158203125
    
**min_conflict:** Consistently fail (10 attempts)


In [45]:
"""
Exercise 1.3
Name: Won Seok Park
Date: Feb/27/2020

Course Scheduling CSP

Notes:
I've used Ian Park's package to set up virtualenv. Import might need a tweak for the code to run on the pycharm

"""

import time

import aima.csp as csp
import aima.search as search

# Define a course scheduling formulation.
def courses_scheduling():
    # Defines the variables and values.
    courses = 'cs108 cs112 cs212 cs214 cs232 cs262 cs344'.split()
    faculty = 'adams vanderlinden plantinga dshuurman norman'.split()
    time_slots = 'mwf9:00-9:50 mwf10:30-11:20 mwf11:30-12:20 mwf12:30-1:20 mwf1:30-2:20'.split()
    classrooms = 'nh253 sb382'.split()
    
    # Set course as a variable and other three as a domain
    variables = courses
    values = faculty + time_slots + classrooms

    # All possible permutations of faculty, time slot, and classroom for domain.
    permut_list = []

    for faculty in faculty:
        for timeslots in time_slots:
            for classroom in classrooms:
                permutation = faculty + ' ' + timeslots + ' ' + classroom
                permut_list.append(permutation)

    # Defines the domain {courses: values}
    domain = {}
    for var in variables:
        domain[var] = permut_list

    # Define neighbors of each variable.
    neighbors = csp.parse_neighbors("""cs108: cs112 cs212 cs214 cs232 cs262 cs344;
                cs112: cs108 cs212 cs214 cs232 cs262 cs344; 
                cs212: cs108 cs112 cs214 cs232 cs262 cs344; 
                cs214: cs108 cs112 cs212 cs232 cs262 cs344;
                cs232: cs108 cs112 cs212 cs214 cs262 cs344; 
                cs262: cs108 cs112 cs212 cs214 cs232 cs344; 
                cs344: cs108 cs112 cs212 cs214 cs232 cs262""")
    

    # Define the constraints on the variables.
    # A and B represents two keys, a and b represents values
    def scheduling_constraint(A, a, B, b):

        # Split "a" and "b" from the permutations for the comparison.
        a_compare = str(a).split()
        b_compare = str(b).split()

        # Check for the overlapping values
        if a_compare[0] == b_compare[0] and a_compare[1] == b_compare[1]:
            return False
        if a_compare[1] == b_compare[1] and a_compare[2] == b_compare[2]:
            return False

        # If no constraint violations, return true.
        return True
        raise Exeption('error')

    return csp.CSP(variables, domain, neighbors, scheduling_constraint)


# Assign problem definition to variable.
problem = courses_scheduling()

""" Select a AIMA search algorithm to use to solve the problem and time its runtime. """
startTime = time.time()

# result = search.depth_first_graph_search(problem)
# result = csp.AC3(problem)
result = csp.backtracking_search(problem)
# result = csp.min_conflicts(problem, max_steps=100000)

endTime = time.time() - startTime

""" A CSP solution printer copied from csp.py. and modified for course scheduling. """


def print_solution(my_results):
    print("\nSolution: " + str(my_results))
    split_results = str(my_results).split()

    print("\nTime taken to find solution: " + str(endTime))


""" Print the solution (or lack thereof). """

if problem.goal_test(problem.infer_assignment()):
    print("Solution:\n")
    print_solution(result)
else:
    print("\nfailed...")
#     print(problem.curr_domains)
#     problem.display(problem.infer_assignment())

    print("\nTime: " + str(endTime))


Solution:


Solution: {'cs108': 'adams mwf9:00-9:50 nh253', 'cs112': 'adams mwf10:30-11:20 nh253', 'cs212': 'adams mwf11:30-12:20 nh253', 'cs214': 'adams mwf12:30-1:20 nh253', 'cs232': 'adams mwf1:30-2:20 nh253', 'cs262': 'vanderlinden mwf9:00-9:50 sb382', 'cs344': 'vanderlinden mwf10:30-11:20 sb382'}

Time taken to find solution: 0.003010988235473633


## Traveling Salesman Local Search Problem Formulation:


**State:** interconnected cities with no city connected to more than two neighbor cities but all cities are connected to ultimately form a ring.

**Action:** two cities were randomly paired to be swapped at the result()


In [31]:
"""
Exercise 1.2
Name: Won Seok Park
Date: Feb/27/2020

Travelling Salesman Problem

Notes:
I've used Ian Park's package to set up virtualenv. Import might need a tweak for the code to run on the pycharm

"""

import math
import itertools
import time
import random

# Import the search algorithms we will use, cooling function, etc.
from aima.search import Problem, hill_climbing, simulated_annealing, exp_schedule


class Salesman(Problem):
    """ Initialize class variables. """

    def __init__(self, distances, initial):
        self.distances = distances
        self.initial = initial

    """ Swap the position of two cities at random. """

    def actions(self, state):
        # Store possible actions.
        actions = []
        
        # Randomly pick cities to swap based on their indices.
        for i in range(0, 50):
            # Selects pairs of cities to be swapped
            my_sample = random.sample(range(len(state)), 2)
            actions.append(my_sample)

        return actions

    def result(self, state, action):
        # Create copy of a state
        new_state = state

        # Convert to list as sets don't support subscripting.
        new_state_list = list(new_state)
        state_list = list(state)

        # Does the actual swapping.
        new_state_list[int(action[0])] = state_list[int(action[1])]
        new_state_list[int(action[1])] = state_list[int(action[0])]
        
        # converted to set to update a state
        return set(new_state_list)

    """ Compute the total distance of the circuit. """

    def value(self, state):

        # Convert to list as sets don't support subscripting.
        state_list = list(state)

        # Determine the pairs of cities in circuit to determine distances for each.
        city_pairs = []

        for i in range(len(state) - 1):

            first = state_list[i]
            second = state_list[i + 1]

            city_pairs.append((str(first), str(second)))

        # So that we take into account the distance from last city back to the starting city.
        back_to_origin = (str(state_list[len(state_list) - 1]), str(state_list[0]))
        city_pairs.append(back_to_origin)

        # Store the distance of the circuit.
        total_distance = 0

        # Calculate the total distance by summing distances for each pair of cities.
        for pairs in city_pairs:
            for _key, _value in self.distances.items():
                if str(pairs) == str(_key):
                    total_distance += _value

        return total_distance


# Create imaginary random location for two cities and compute distance
def get_dist():
    x1 = random.randint(-100, 100)
    x2 = random.randint(-100, 100)
    y1 = random.randint(-100, 100)
    y2 = random.randint(-100, 100)
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)


""" Attempt to solve a sample Traveling Salesman Problem and print solution to console. """

if __name__ == '__main__':

    # Create simple list of cities to travel to.
    cities = {'King', 'Queen', 'Bishop', 'Knight', 'Rook', 'Pawn1', 'Pawn2', 'Pawn3', 
              'Pawn4', 'Pawn5', 'Pawn6', 'Pawn7', 'Pawn8'}

    # Create dictionary of distances between all possible pairs of cities A --> B only).
    city_pairs = itertools.combinations(cities, 2)
    city_dist = {}
    for element in city_pairs:
        city_dist[element] = get_dist()

        reversedTuple = (element[1], element[0])
        # Distance between cities B --> A
        city_dist[reversedTuple] = city_dist[element]

    # Initialize the Traveling Salesman Problem.
    travel = Salesman(city_dist, cities)

    # Solve the problem using hill climbing.
    start_time = time.time()
    hill_solution = hill_climbing(travel)
    end_time = time.time() - start_time
    print('Hill-climbing:')
    print('\tSolution: ' + str(hill_solution))
    print('\tValue:    ' + str(travel.value(hill_solution)))
    print('\tTime:     ' + str(end_time))

    # Solve the problem using simulated annealing.
    start_time = time.time()
    annealing_solution = simulated_annealing(travel,
                                             exp_schedule(k=20, lam=0.005, limit=10000))
    end_time = time.time() - start_time
    print('Simulated annealing:')
    print('\tSolution: ' + str(annealing_solution))
    print('\tValue:    ' + str(travel.value(annealing_solution)))
    print('\tTime:     ' + str(end_time))


Hill-climbing:
	Solution: {'Pawn4', 'Pawn6', 'Pawn7', 'Pawn8', 'Pawn1', 'Pawn2', 'Rook', 'Knight', 'Queen', 'Bishop', 'King', 'Pawn5', 'Pawn3'}
	Value:    1688.2519855809787
	Time:     0.22320771217346191
Simulated annealing:
	Solution: {'Pawn4', 'Pawn8', 'Pawn7', 'Rook', 'Pawn6', 'Pawn2', 'Pawn1', 'Knight', 'Queen', 'Bishop', 'King', 'Pawn5', 'Pawn3'}
	Value:    1689.0343084157203
	Time:     31.323342084884644


Both simulated annealing and hill climbing managed to find a solution to the traveling salesman problem.

Hill-climbing:
	Solution: {'Pawn4', 'Pawn8', 'Rook', 'Pawn6', 'Pawn1', 'Pawn2', 'Pawn7', 'Queen', 'Knight', 'Bishop', 'King', 'Pawn5', 'Pawn3'}
	Value:    1616.7739870267326
	Time:     0.25135254859924316
Simulated annealing:
	Solution: {'Pawn4', 'Pawn8', 'Rook', 'Pawn6', 'Pawn1', 'Pawn2', 'Pawn7', 'Knight', 'Queen', 'Bishop', 'King', 'Pawn5', 'Pawn3'}
	Value:    1527.6888248954217
	Time:     31.392478942871094

Hill-climbing is magnitude faster than Simulated-annealing when it comes to computational speed. However, Simulated-annealing does better job finding the shortest path, especially when the number of cities grew. For example, both HC and SA consistently got the same value for the city-size of 4. At the city-size of 8, SA got better result after many trials. For the city-size of 12, SA found better solution than HC once every few runs. I expect trends to continue as city-size grows, and at certain point SA would perform much better than HC as more and more local minima get introduced.